In [1]:
from utils.molec_class_files import r14, r32, r50, r125, r134a, r143a, r170, r41, r23, r161, r152a, r152, r134, r143, r116
from utils import atom_type, opt_atom_types
import numpy as np
import unyt as u
import pandas as pd
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real, generate_lhs
from fffit.fffit.plot import plot_obj_contour
import os

2024-06-28 14:49:10.305961: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-28 14:49:10.349052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 14:49:10.349096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-28 14:49:10.350721: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-28 14:49:10.358369: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-28 14:49:10.359492: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [10]:
import scipy 
import os
repeats = 1
seed = 1
#Get obj from a set of parameters
at_class = 11
save_data = False
obj_choice = "ExpVal"
obj_choice_p = "ExpValPrior"
molec_names = ["R14", "R32", "R50", "R125", "R134a", "R143a", "R170"]
at_optimizer =  opt_atom_types.Opt_ATs(molec_names, at_class, repeats, seed, obj_choice_p)
# visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, obj_choice)
# visual.check_GPs()
setup = opt_atom_types.Problem_Setup(molec_names, at_class, obj_choice)
all_molec_dir = setup.use_dir_name
all_df = pd.read_csv(all_molec_dir /"best_per_run.csv", header = 0)
sorted_df = all_df.sort_values(by='Min Obj', ascending = True).reset_index(drop = True)
first_param_name = setup.at_class.at_names[0] + "_min"
last_param_name = setup.at_class.at_names[-1] + "_min"
best_pref = all_df.loc[0, first_param_name:last_param_name].values
best_real= at_optimizer.values_pref_to_real(best_pref)
# best_real = generate_lhs(1, at_optimizer.at_class.at_bounds_nm_kjmol, at_optimizer.seed, labels = None)
obj = at_optimizer.calc_obj(best_real)
print(obj[0])

# #Get Jac and Hess info from a set of parameters
# dir_name = at_optimizer.make_results_dir(list(at_optimizer.molec_data_dict.keys()))
# save_path = os.path.join(dir_name, "best_set" + "_jac_approx.npy")
# hess = np.load(save_path)
# print(hess)
# jac = at_optimizer.approx_jac(best_real)
# hess = at_optimizer.approx_hess(best_real)
# print(scipy.linalg.norm(jac, ord = np.inf))
# print(jac)
# print(hess)
# eigval, eigvec = scipy.linalg.eig(hess)
# print(eigval)

/scratch365/mcarlozo/generalizedFF/Results/at_11/R14-R32-R50-R170-R125-R134a-R143a/ExpVal/best_per_run.csv
best_info:       Run  Iter      Min Obj  sigma_Cm_min  sigma_C1_min  sigma_C2_0_min  \
0     50   490  8917.121865      3.823961      3.399209        3.854654   
1     71   538  8918.358766      3.827549      3.397984        3.855499   
2    157   587  8918.410867      3.825338      3.397634        3.855525   
3     60   615  8918.437477      3.825774      3.397945        3.854526   
4    156   630  8918.584778      3.826194      3.398637        3.855615   
..   ...   ...          ...           ...           ...             ...   
195   70   208  9672.588431      3.827142      3.396242        3.852360   
196   12   147  9684.986241      3.830312      3.391409        3.850863   
197  191   190  9706.780385      3.830707      3.397159        3.851860   
198   95   156  9714.818106      3.832500      3.393837        3.852295   
199   20   125  9850.549130      3.823841      3.387793 